In [95]:
%load_ext autoreload
%autoreload 2
import torch
#from QTensorAI.qtensor_ai.Quantum_Neural_Net import QNN
from QTensorAI.qtensor_ai.ParallelQTensor import ParallelTorchQkernelComposer, TraceEvaluationCircuitComposer, RPQCComposer, ParallelQtreeSimulator, ParallelTorchBackend, PauliPhase

#from guppy import hpy; h=hpy()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
qnn = QNN(10, 1, variational_layers=2)
qnn.eval()

Using previously saved contraction order at  <_io.BufferedReader name='./Saved_Contraction_Orders/GreedyOptimizer/Qkernel/n_qubits_10_n_layers_2.pickle'>


QNN()

In [3]:
x = torch.rand((10, 10))
y = qnn(x)
loss = y.mean()
loss.backward()

NameError: name 'qnn' is not defined

In [116]:
in_features = 2
variational_layers = 2
n_batch = 10
device = 'cpu'
com = ParallelTorchQkernelComposer(in_features)
rpqc1 = RPQCComposer(in_features)
rpqc2 = RPQCComposer(in_features)
tcom = TraceEvaluationCircuitComposer(in_features, RPQCComposer.name())

init_inputs = torch.rand(n_batch, in_features, requires_grad=False).to(device)
init_params1 = torch.rand(n_batch, in_features, variational_layers, requires_grad=False).to(device)
init_params2 = torch.rand(n_batch, in_features, variational_layers, requires_grad=False).to(device)
gate_types1 = torch.randint(3, (n_batch*in_features*variational_layers,))
gate_types1 = gate_types1.reshape((n_batch, variational_layers, in_features))
gate_types2 = torch.randint(3, (n_batch*in_features*variational_layers,))
gate_types2 = gate_types2.reshape((n_batch, variational_layers, in_features))

#com.expectation_circuit(init_inputs, init_params1)
#target_circuit = com.static_circuit#+com.static_circuit
rpqc1.expectation_circuit(gate_types1, init_params1)
rpqc2.expectation_circuit(gate_types2, init_params2)
#tcom.expectation_circuit(target_circuit + tcom.inverse(target_circuit))
tcom.expectation_circuit(rpqc1.static_circuit + rpqc2.static_circuit)
tcom.static_circuit

False
False
tensor([[ 0.7071,  0.7071],
        [ 0.7071, -0.7071]])


[H(2),
 cX(2,0),
 H(3),
 cX(3,1),
 H(0),
 H(1),
 PauliPhase[alpha=0.78,gates=2.00](0),
 PauliPhase[alpha=0.54,gates=1.00](1),
 cX(0,1),
 PauliPhase[alpha=0.36,gates=1.00](0),
 PauliPhase[alpha=0.05,gates=2.00](1),
 cX(0,1),
 H(0),
 H(1),
 PauliPhase[alpha=0.84,gates=2.00](0),
 PauliPhase[alpha=0.05,gates=1.00](1),
 cX(0,1),
 PauliPhase[alpha=0.09,gates=1.00](0),
 PauliPhase[alpha=0.65,gates=2.00](1),
 cX(0,1),
 cX+(3,1),
 H+(3),
 cX+(2,0),
 H+(2)]

In [121]:
sim = ParallelQtreeSimulator(backend=ParallelTorchBackend())
sim.simulate_batch(tcom.static_circuit)

tensor([-0.0841+0.0000j, -0.0413+0.0000j, -0.2258+0.0990j, -0.0622+0.0707j,
         0.0220-0.0949j,  0.2220+0.1625j, -0.1128+0.0559j, -0.0333-0.3308j,
        -0.0534-0.2260j,  0.1606+0.0866j])

In [122]:
import qiskit